In [25]:
! pip install kaleido

In [26]:
!pip install tensorflow

In [1]:
import numpy as np
import os
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tarfile
import zipfile
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import AgglomerativeClustering
from scipy.io import loadmat
from urllib.request import urlretrieve
from os.path import isfile, isdir
import plotly.figure_factory as ff
import plotly.express as px
import statsmodels.api as sm
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import glob
import random
import matplotlib.image as img
from PIL import Image
#Install pillow
from PIL import Image
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [129]:
train_folder = 'datos/train'
categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen',
              'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse',
              'Small-flowered Cranesbill', 'Sugar beet']

In [130]:
images = []
labels = []

In [131]:
for category in categories:
    folder_path = os.path.join(train_folder, category)
    # Se itera sobre los archivos de imagen en cada carpeta
    for file_name in os.listdir(folder_path):
        # Se lee la imagen y convertirla a escala de grises
        image_path = os.path.join(folder_path, file_name)
        image = Image.open(image_path).convert('L')
        # Se redimensiona la imagen a un tamaño fijo, por ejemplo (128, 128)
        image = image.resize((128, 128))
        # Se convierte la imagen a un array numpy
        image_array = np.array(image)
        # Se agrega la imagen y la etiqueta a las listas
        images.append(image_array)
        labels.append(category)

images = np.array(images)
labels = np.array(labels)

In [132]:
np.array(images).shape

(4750, 128, 128)

In [133]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [134]:
labels_categorical = to_categorical(labels_encoded)

In [135]:
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.3, random_state=42)

In [136]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [137]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(categories), activation='softmax'))

In [138]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [139]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
52/52 [==============================] - 23s 402ms/step - loss: 2.4531 - accuracy: 0.1293 - val_loss: 2.4310 - val_accuracy: 0.1389
Epoch 2/10
52/52 [==============================] - 26s 511ms/step - loss: 2.4104 - accuracy: 0.1492 - val_loss: 2.3979 - val_accuracy: 0.1396
Epoch 3/10
52/52 [==============================] - 26s 509ms/step - loss: 2.3465 - accuracy: 0.1645 - val_loss: 2.3287 - val_accuracy: 0.1691
Epoch 4/10
52/52 [==============================] - 27s 518ms/step - loss: 2.3021 - accuracy: 0.1892 - val_loss: 2.3155 - val_accuracy: 0.1775
Epoch 5/10
52/52 [==============================] - 27s 512ms/step - loss: 2.2746 - accuracy: 0.1979 - val_loss: 2.3013 - val_accuracy: 0.1719
Epoch 6/10
52/52 [==============================] - 27s 523ms/step - loss: 2.2136 - accuracy: 0.2238 - val_loss: 2.2710 - val_accuracy: 0.2063
Epoch 7/10
52/52 [==============================] - 27s 529ms/step - loss: 2.1563 - accuracy: 0.2553 - val_loss: 2.2413 - val_accuracy: 0.2218

In [140]:
test_folder = 'datos/test'
test_images = []

In [141]:
for file_name in os.listdir(test_folder):
    image_path = os.path.join(test_folder, file_name)
    image = Image.open(image_path).convert('L')
    image = image.resize((128, 128))
    image_array = np.array(image)
    test_images.append(image_array)

In [142]:
test_images = np.array(test_images) / 255.0

In [143]:
predictions = model.predict(test_images)

25/25 [==============================] - 1s 50ms/step


In [144]:
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

In [145]:
results_df = pd.DataFrame({'file': os.listdir(test_folder), 'species': predicted_labels})

In [146]:
predictions_file = os.path.join('datos', 'predictions.csv')
results_df.to_csv(predictions_file, index=False)

In [147]:
predictions_data = pd.read_csv(predictions_file)
print(predictions_data)

              file                    species
0    0021e90e4.png  Small-flowered Cranesbill
1    003d61042.png           Loose Silky-bent
2    007b3da8b.png           Common Chickweed
3    0086a6340.png           Common Chickweed
4    00c47e980.png                    Fat Hen
..             ...                        ...
789  fea355851.png          Scentless Mayweed
790  fea3da57c.png                    Fat Hen
791  fef2ade8c.png           Common Chickweed
792  ff65bc002.png           Common Chickweed
793  ffc6f8527.png           Common Chickweed

[794 rows x 2 columns]
